In [39]:
import openai
import os
import pandas as pd 
os.environ["OPENAI_API_KEY"] = ""

In [40]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [70]:
df = pd.read_csv('./sales_data_sample.csv')



In [58]:


# Create a completion using the ChatCompletion API
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
#     {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."},
    
#   ]
  messages = [
      {
          "role":"user", "content" : "what is capital of the andhra pradesh state in india in 2024"
      }
  ], 
  max_tokens = 60,
  temperature = 0.1
)

# Print the completion
response.choices[0].message['content']
# print(completion.choices[0].message['content'])


'The capital of Andhra Pradesh state in India in 2024 is likely to still be Amaravati. However, it is important to note that political decisions and changes can impact the capital city, so it is always best to verify the current capital at the time of reference.'

In [61]:
import sqlalchemy
from sqlalchemy import create_engine 
from sqlalchemy import text 

In [62]:
# create the temp database in the ram 
# push the dataframe into the temp database 

# sql query on the temp db 


temp_db = create_engine('sqlite:///:memory', echo=True)

In [64]:
# it creates a table sales in the database 
data = df.to_sql(name='Sales', con=temp_db)

2024-05-28 16:16:44,038 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-28 16:16:44,039 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2024-05-28 16:16:44,040 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-28 16:16:44,043 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2024-05-28 16:16:44,043 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-28 16:16:44,044 INFO sqlalchemy.engine.Engine ROLLBACK
2024-05-28 16:16:44,045 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-28 16:16:44,046 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	

In [67]:
with temp_db.connect() as conn : 
    # makes the connection 
    # here we can run the sql connection 
    result = conn.execute(text("select sum(sales) from Sales"))
    #auto close the connection 

2024-05-28 16:19:49,217 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-28 16:19:49,221 INFO sqlalchemy.engine.Engine select sum(sales) from Sales
2024-05-28 16:19:49,222 INFO sqlalchemy.engine.Engine [generated in 0.00460s] ()
2024-05-28 16:19:49,224 INFO sqlalchemy.engine.Engine ROLLBACK


In [68]:
result.all()

[(10032628.85000001,)]

In [80]:
# function to create the prompt structure.

def create_table_definition(df):
    prompt = """### sqlite sql table, with its properties
    #
    # Sales({})
    #
    
    """.format(",".join(str(col) for col in df.columns))
    return prompt 

print(create_table_definition(df))

### sqlite sql table, with its properties
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
    #
    
    


In [81]:
def prompt_input():
    nlp_text = input('enter the info you want :')
    return nlp_text

In [82]:
def combine_prompts(df, query_prompt):
    definition = create_table_definition(df)
    query_init_string = f"### A query to answer :{query_prompt}\nSELECT"
    return definition + query_init_string

In [85]:
nlp_text = prompt_input() 

prompt = combine_prompts(df, nlp_text)

prompt

enter the info you want :can you sum up the sales per each quarter


'### sqlite sql table, with its properties\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n    #\n    \n    ### A query to answer :can you sum up the sales per each quarter\nSELECT'

In [104]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
    ],
    max_tokens=100,
    temperature=0.5
)

# print(response.choices[0].message['content'].strip())
response_content = response.choices[0].message['content'].strip()
def extract_sql_query(response_content):
    start_marker = "```sql"
    end_marker = "```"
    
    start_index = response_content.find(start_marker)
    if start_index != -1:
        start_index += len(start_marker)
        end_index = response_content.find(end_marker, start_index)
        if end_index != -1:
            return response_content[start_index:end_index].strip()
    return response_content


final_query = extract_sql_query(response_content)

In [109]:
with temp_db.connect() as conn : 
    # makes the connection 
    # here we can run the sql connection 
    result = conn.execute(text(final_query))
    #auto close the connection 
    
    


2024-05-28 17:14:30,103 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-28 17:14:30,105 INFO sqlalchemy.engine.Engine SELECT QTR_ID, SUM(SALES) AS TOTAL_SALES
FROM Sales
GROUP BY QTR_ID;
2024-05-28 17:14:30,106 INFO sqlalchemy.engine.Engine [cached since 46.13s ago] ()
2024-05-28 17:14:30,108 INFO sqlalchemy.engine.Engine ROLLBACK


In [110]:
result.all()

[(1, 2350817.7300000004),
 (2, 2048120.2999999986),
 (3, 1758910.8099999994),
 (4, 3874780.01)]